In [24]:
import pandas as pd
import numpy as np 
from openai import OpenAI
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity 
import pickle
import re

In [25]:
openai_api_key = ''

In [64]:
client = OpenAI(api_key= openai_api_key)
articles_df = pd.DataFrame(pd.read_csv('../data/Result_2.csv')) 
articles_df

,title,tags,category
0,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",INVESTMENT
1,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,"부동산,재테크,투자",INVESTMENT
2,애들건강 음식\nㅕ,"반나나,채소,삶은고기,애호박,요플레",RELATIONSHIP
3,운동\n,테그,HOBBY
4,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",SIDE_JOB
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",SIDE_JOB
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",SIDE_JOB
7,웹소설을 같이 쓴다는 것,"1등,웹소설,카카오페이지",SIDE_JOB
8,카카오페이지 웹소설 1위 찍다!,"1등,웹소설,카카오페이지,인기작품",SIDE_JOB
9,발로란트,"발로란트,세이지,게임,로터스",GAME


In [65]:
articles_df["processed_text"] = articles_df.apply(lambda x: re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]","",
                                                         f"{x['title']} {' '.join(x['tags'].split(','))}"), axis=1)


In [66]:
stop_words = pd.read_csv('../data/stop_words_ko.csv')
stop_words


,stop_words
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
770,원
771,잘
772,통하
773,소리


In [67]:
from nltk.tokenize import word_tokenize

articles_df["processed_text"] = (
    articles_df.apply(lambda x: ' '.join([word for word in word_tokenize(x['processed_text']) if word not in stop_words['stop_words'].tolist()]), axis=1))

articles_df

,title,tags,category,processed_text
0,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",INVESTMENT,한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...
1,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,"부동산,재테크,투자",INVESTMENT,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자
2,애들건강 음식\nㅕ,"반나나,채소,삶은고기,애호박,요플레",RELATIONSHIP,애들건강 음식 반나나 채소 삶은고기 애호박 요플레
3,운동\n,테그,HOBBY,운동 테그
4,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",SIDE_JOB,웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",SIDE_JOB,웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",SIDE_JOB,웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품
7,웹소설을 같이 쓴다는 것,"1등,웹소설,카카오페이지",SIDE_JOB,웹소설을 쓴다는 1등 웹소설 카카오페이지
8,카카오페이지 웹소설 1위 찍다!,"1등,웹소설,카카오페이지,인기작품",SIDE_JOB,카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품
9,발로란트,"발로란트,세이지,게임,로터스",GAME,발로란트 발로란트 세이지 게임 로터스


In [68]:
def get_embedding(text: str, model="text-embedding-3-large"):
    text = text.replace('\n', ' ')
    return client.embeddings.create(input=[text], model=model).data[0].embedding

articles_df['embedding'] = articles_df.apply(lambda x: get_embedding(x['processed_text']), axis=1)


# articles_df['embedding'] = articles_df.apply(lambda x: get_embedding(f"{x['title']} - {' '.join(x['tags'].split(','))}" ), axis=1)

with open( "embedded-article-with-tag-category-large", "wb" ) as file:
    pickle.dump( articles_df, file)

# with open( "embedded-article-with-tag-content", "rb" ) as file:
#     articles_df = pickle.load(file)
    
articles_df
    

,title,tags,category,processed_text,embedding
0,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",INVESTMENT,한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...,"[0.012517409399151802, -0.04002080112695694, -..."
1,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,"부동산,재테크,투자",INVESTMENT,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자,"[-0.004191316198557615, -0.01968361809849739, ..."
2,애들건강 음식\nㅕ,"반나나,채소,삶은고기,애호박,요플레",RELATIONSHIP,애들건강 음식 반나나 채소 삶은고기 애호박 요플레,"[-0.023416705429553986, -0.017992764711380005,..."
3,운동\n,테그,HOBBY,운동 테그,"[-0.004921027924865484, -0.020684998482465744,..."
4,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",SIDE_JOB,웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...,"[-0.02818647027015686, -0.041322916746139526, ..."
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",SIDE_JOB,웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...,"[0.024982821196317673, -0.05572228133678436, -..."
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",SIDE_JOB,웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품,"[0.001184988534078002, -0.04099058732390404, -..."
7,웹소설을 같이 쓴다는 것,"1등,웹소설,카카오페이지",SIDE_JOB,웹소설을 쓴다는 1등 웹소설 카카오페이지,"[0.0042601414024829865, -0.03769685700535774, ..."
8,카카오페이지 웹소설 1위 찍다!,"1등,웹소설,카카오페이지,인기작품",SIDE_JOB,카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품,"[0.005625215824693441, -0.049295783042907715, ..."
9,발로란트,"발로란트,세이지,게임,로터스",GAME,발로란트 발로란트 세이지 게임 로터스,"[-0.02611675299704075, -0.030597371980547905, ..."


In [69]:
from sklearn.cluster import KMeans 
clusters = KMeans(  random_state=0).fit_predict(articles_df['embedding'].tolist())
articles_df['cluster'] = clusters

articles_df

,title,tags,category,processed_text,embedding,cluster
0,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",INVESTMENT,한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...,"[0.012517409399151802, -0.04002080112695694, -...",2
1,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,"부동산,재테크,투자",INVESTMENT,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자,"[-0.004191316198557615, -0.01968361809849739, ...",2
2,애들건강 음식\nㅕ,"반나나,채소,삶은고기,애호박,요플레",RELATIONSHIP,애들건강 음식 반나나 채소 삶은고기 애호박 요플레,"[-0.023416705429553986, -0.017992764711380005,...",4
3,운동\n,테그,HOBBY,운동 테그,"[-0.004921027924865484, -0.020684998482465744,...",7
4,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",SIDE_JOB,웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...,"[-0.02818647027015686, -0.041322916746139526, ...",1
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",SIDE_JOB,웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...,"[0.024982821196317673, -0.05572228133678436, -...",1
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",SIDE_JOB,웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품,"[0.001184988534078002, -0.04099058732390404, -...",1
7,웹소설을 같이 쓴다는 것,"1등,웹소설,카카오페이지",SIDE_JOB,웹소설을 쓴다는 1등 웹소설 카카오페이지,"[0.0042601414024829865, -0.03769685700535774, ...",1
8,카카오페이지 웹소설 1위 찍다!,"1등,웹소설,카카오페이지,인기작품",SIDE_JOB,카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품,"[0.005625215824693441, -0.049295783042907715, ...",1
9,발로란트,"발로란트,세이지,게임,로터스",GAME,발로란트 발로란트 세이지 게임 로터스,"[-0.02611675299704075, -0.030597371980547905, ...",6


In [79]:
query_test_description =  word_tokenize(re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]","","돈 벌고 싶은데")) 
query_test_description = ' '.join([word for word in query_test_description if not word in stop_words])
query_test_description

'돈 벌고 싶은데'

In [80]:
 

query_embedding = get_embedding(query_test_description)

articles_df['similarity'] = articles_df['embedding'].apply(lambda x: cosine_similarity(np.array(x).reshape(1, -1), np.array(query_embedding).reshape(1, -1))[0][0])
selected_df = articles_df.sort_values(by='similarity', ascending=False)
 
selected_df

,title,tags,category,processed_text,embedding,cluster,similarity
0,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",INVESTMENT,한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...,"[0.012517409399151802, -0.04002080112695694, -...",2,0.452688
17,맥을 똑똑하게쓰기,"맥북,돈값해야지,맥북비싸",HOBBY,맥을 똑똑하게쓰기 맥북 돈값해야지 맥북비싸,"[0.022383540868759155, -0.01917628012597561, -...",5,0.365842
14,"조직 밖에서 스스로 돈벌기, 절반의 성공과 절반의 실패","일러스트레이터,부업,자기계발,N잡,커리어,수익화,디자이너,자기개발,프리랜서",CAREER,조직 밖에서 스스로 돈벌기 절반의 성공과 절반의 실패 일러스트레이터 부업 자기계발 ...,"[-0.00323639833368361, -0.006465445272624493, ...",1,0.355385
22,웹소설 진단 서비스 - 카카오페이지 1등이 웹소설로 돈 버는 법 알려드림,웹소설,SIDE_JOB,웹소설 진단 서비스 카카오페이지 1등이 웹소설로 돈 버는 법 알려드림 웹소설,"[0.0067979260347783566, -0.05656076967716217, ...",1,0.339463
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",SIDE_JOB,웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...,"[0.024982821196317673, -0.05572228133678436, -...",1,0.336909
27,직장인이 부동산 부자되는 궁극의 7단계 전략,"직장인투자,부동산,부동산강의,투자관리,부자되는법,투자방법,부동산투자,부자되기",INVESTMENT,직장인이 부동산 부자되는 궁극의 7단계 전략 직장인투자 부동산 부동산강의 투자관리 ...,"[0.020873481407761574, -0.04385462775826454, -...",2,0.295455
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",SIDE_JOB,웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품,"[0.001184988534078002, -0.04099058732390404, -...",1,0.295218
23,직장인 부업 추천 - 블로그를 시작하세요 :),"직장인부업,블로그,부업,네이버블로그",SIDE_JOB,직장인 부업 추천 블로그를 시작하세요 직장인부업 블로그 부업 네이버블로그,"[0.02812519483268261, -0.031949013471603394, -...",2,0.285249
1,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,"부동산,재테크,투자",INVESTMENT,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자,"[-0.004191316198557615, -0.01968361809849739, ...",2,0.279652
16,스타벅스 꿀팁이 궁금하세요?,"스타벅스,별적립",ETC,스타벅스 꿀팁이 궁금하세요 스타벅스 별적립,"[-0.003289257874712348, -0.04253232106566429, ...",5,0.272597
